In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

IMG_SIZE = 224
BATCH = 32

(ds_train, ds_test), info = tfds.load(
    "rock_paper_scissors",
    split=["train", "test"],
    as_supervised=True,
    with_info=True
)

num_classes = info.features["label"].num_classes
print("classes:", num_classes)

# preprocessing
def preprocess(img, label):
    img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))
    img = tf.cast(img, tf.float32) / 255.0
    return img, label

ds_train = (ds_train
            .map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
            .shuffle(1000)
            .batch(BATCH)
            .prefetch(tf.data.AUTOTUNE))

ds_test = (ds_test
           .map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
           .batch(BATCH)
           .prefetch(tf.data.AUTOTUNE))

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/rock_paper_scissors/incomplete.DS5CQX_3.0.0/rock_paper_scissors-train.tfre…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/rock_paper_scissors/incomplete.DS5CQX_3.0.0/rock_paper_scissors-test.tfrec…

Dataset rock_paper_scissors downloaded and prepared to /root/tensorflow_datasets/rock_paper_scissors/3.0.0. Subsequent calls will reuse this data.
classes: 3


In [ ]:
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2

base = MobileNetV2(weights="imagenet", include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
base.trainable = False  # Freeze

inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = base(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)
model = models.Model(inputs, outputs)

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

history1 = model.fit(ds_train, validation_data=ds_test, epochs=5)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/5
79/79 ━━━━━━━━━━━━━━━━━━━━ 73s 859ms/step - accuracy: 0.3837 - loss: 1.2199 - val_accuracy: 0.5161 - val_loss: 1.0844
Epoch 2/5
79/79 ━━━━━━━━━━━━━━━━━━━━ 65s 818ms/step - accuracy: 0.7241 - loss: 0.7299 - val_accuracy: 0.6505 - val_loss: 0.8673
Epoch 3/5
79/79 ━━━━━━━━━━━━━━━━━━━━ 65s 821ms/step - accuracy: 0.8714 - loss: 0.4971 - val_accuracy: 0.7715 - val_loss: 0.7181
Epoch 4/5
79/79 ━━━━━━━━━━━━━━━━━━━━ 85s 866ms/step - accuracy: 0.9365 - loss: 0.3557 - val_accuracy: 0.8118 - val_loss: 0.6384
Epoch 5/5
79/79 ━━━━━━━━━━━━━━━━━━━━ 82s 859ms/step - accuracy: 0.9477 - loss: 0.2835 - val_accuracy: 0.8306 - val_loss: 0.5648


In [ ]:
base.trainable = True

for layer in base.layers[:-20]:
    layer.trainable = False

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

In [ ]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss", patience=3, restore_best_weights=True
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.2, patience=2, min_lr=1e-7
    )
]

In [ ]:
history2 = model.fit(
    ds_train,
    validation_data=ds_test,
    epochs=10,
    callbacks=callbacks
)

Epoch 1/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 88s 1s/step - accuracy: 0.8437 - loss: 0.3985 - val_accuracy: 0.7688 - val_loss: 0.5593 - learning_rate: 1.0000e-05
Epoch 2/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 87s 1s/step - accuracy: 0.9959 - loss: 0.0549 - val_accuracy: 0.7688 - val_loss: 0.5471 - learning_rate: 1.0000e-05
Epoch 3/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 139s 1s/step - accuracy: 0.9985 - loss: 0.0257 - val_accuracy: 0.7823 - val_loss: 0.5302 - learning_rate: 1.0000e-05
Epoch 4/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 140s 1s/step - accuracy: 0.9998 - loss: 0.0180 - val_accuracy: 0.7930 - val_loss: 0.5159 - learning_rate: 1.0000e-05
Epoch 5/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 89s 1s/step - accuracy: 0.9996 - loss: 0.0120 - val_accuracy: 0.8172 - val_loss: 0.4890 - learning_rate: 1.0000e-05
Epoch 6/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 88s 1s/step - accuracy: 1.0000 - loss: 0.0077 - val_accuracy: 0.8441 - val_loss: 0.4625 - learning_rate: 1.0000e-05
Epoch 7/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 142s 1s/step - accuracy: 1.0000 - lo

In [ ]:
test_loss, test_acc = model.evaluate(ds_test)
print("Test accuracy:", test_acc)
print("Test loss:", test_loss)

12/12 ━━━━━━━━━━━━━━━━━━━━ 9s 750ms/step - accuracy: 0.8855 - loss: 0.4158
Test accuracy: 0.8844085931777954
Test loss: 0.40893223881721497


In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

y_true = np.concatenate([y for x, y in ds_test], axis=0)
y_pred = np.argmax(model.predict(ds_test), axis=1)

print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred))

12/12 ━━━━━━━━━━━━━━━━━━━━ 11s 786ms/step
[[124   0   0]
 [ 43  81   0]
 [  0   0 124]]
              precision    recall  f1-score   support

           0       0.74      1.00      0.85       124
           1       1.00      0.65      0.79       124
           2       1.00      1.00      1.00       124

    accuracy                           0.88       372
   macro avg       0.91      0.88      0.88       372
weighted avg       0.91      0.88      0.88       372

